# Plot Attrib Perf and historical risk

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>

The raw code for this IPython notebook is by default hidden for easier reading.To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.

In [1]:
from IPython.display import display, HTML

In [3]:
%matplotlib inline

#import matplotlib
#matplotlib.use('Qt5Agg')

In [4]:
# import standard libs
from IPython.display import display
#from IPython.core.debugger import set_trace as bp
#import datetime

# import in-house lib
import updPosesHT4 as upd

# import python scientific stack
import pandas as pd
import numpy as np
#import datetime

import matplotlib.pyplot  as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('talk')

# import visual tools
#import bqplot as bq
#import plotly.graph_objs as go
#import plotly
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)

# import util libs
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox



Toggle output scrolling of selected cells

In [5]:
start_date, end_date = pd.datetime(2017,1,1), pd.datetime.today()

In [6]:
print('Loading Data from {:%d,%b %Y} to {:%d,%b %Y}'.format(start_date, end_date))
df = upd.getPosesRpt(start_date, end_date, 1.0).reset_index()
df = df.drop(df[df.PM=='PTF'].index)

Loading Data from 01,Jan 2017 to 17,Jul 2018


In [7]:
def wSelectMultiple(df, fields):
    return {fld: widgets.SelectMultiple(options= ['ALL'] + elts, 
                                        value= ['ALL'],
                                        rows= min(len(elts)+1, 8),
                                        description= fld)
                    for fld, elts in 
                        ((fld, list(df[fld].sort_values().unique())) for fld in fields)}

In [8]:
flds = ['Asset', 'PM', 'PtfId', 'Code']
wSel = wSelectMultiple(df.reset_index(), flds)
wSel['FundReference'] = widgets.ToggleButtons(options= ['DGA', 'ITX3'])
wFields = widgets.ToggleButtons(options=['pl', 'rv', 'dir', 'deltaAUM', 'gammaAUM', 'vegaAUMbp', 'thetaAUMbp', 'bpvEur'],
    description='Fields',
    tooltips=['PL total in bps', 'PL Relative Value in bps of slow', 'PL eq. benchmark in bps'],)

In [9]:
def update_Code(*args):
    #print(args)
    codes = list(df[df['Asset'].isin(list(args[0]["new"]))].Code.sort_values().unique())
    #codes = list(df.query(f'Asset in {list(args[0]["new"])}').Code.unique())
    wSel['Code'].options = ['ALL'] + codes
    wSel['Code'].value = ['ALL']   
wSel['Asset'].observe(update_Code, 'value')

In [10]:
%matplotlib -l

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'wx', 'qt', 'qt4', 'qt5', 'osx', 'nbagg', 'notebook', 'agg', 'inline', 'ipympl']


In [11]:
# @hidden_cell
def plot_filtered(field= wFields, **kwargs):
    global select, select0
    select = df
    flds = {fld: values for fld, values in kwargs.items() if not 'ALL' in values}
    #print(flds)
    for fld, values in flds.items():
        if isinstance(values, str):
            values = [values]
        if 'mask' in locals(): 
            mask = mask & df[fld].isin(values)
        else:
            mask = df[fld].isin(values)
    if 'mask' in locals():
        select = df[mask]
        gp = ['PtfDate'] + list(flds.keys())
    else:
        gp = ['PtfDate']
    select = select.groupby(gp)[wFields.value].sum()
    if flds:
        select = select.unstack(0).T
        if wFields.value in ['pl', 'rv', 'dir']:
            select= select.cumsum()
    if select.empty or not select.any().any():
        plt.gca().remove();
    else:
        if 'Code' in select.columns.names:
            # summed by codes
            select0 = select.stack('Code').groupby('PtfDate').sum()
            ax0= select0.plot(figsize= (16, 9), title= wFields.value + f', as of {select.index[-1]:%d %b %Y}')
            ax0.yaxis.tick_right()
            ax0.hlines(0, *ax0.xaxis.get_data_interval(), linewidth = 2.0, color='r');        
        # Details by codes
        ax= select.plot(figsize= (16, 9), title= wFields.value + f' (detailed by Code), as of {select.index[-1]:%d %b %Y}')
        ax.yaxis.tick_right()
        ax.hlines(0, *ax.xaxis.get_data_interval(), linewidth = 2.0, color='r');
        
    return kwargs
w = widgets.interactive(plot_filtered, **wSel)

In [12]:
display(VBox([wFields, 
              widgets.Label('Please select filters'),
              #HBox(w.children[:-1]), 
              HBox(list(wSel.values())), 
              w.children[-1]]))

You can view detail of the plot in the Dataframe "select":

In [30]:
# tail of the plot
if 'select' in globals():
    display(round(select.fillna(method= 'pad').tail(), 1))

PM,CHC
FundReference,DGA
PtfDate,
2018-07-10,370.6
2018-07-11,355.1
2018-07-12,356.7
2018-07-13,362.2
2018-07-16,367.8


In [52]:
!jupyter nbconvert --help

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

In [46]:
!dir

 Volume in drive C is OSDisk
 Volume Serial Number is EC8E-52D6

 Directory of C:\Users\fm391.CORP\GMdev\TP\ipywidgets

17/07/2018  13:27    <DIR>          .
17/07/2018  13:27    <DIR>          ..
16/07/2018  15:03    <DIR>          .ipynb_checkpoints
12/07/2018  14:57            77 518 download.png
11/07/2018  22:20        13 327 027 getPosesRpt1.pkl
11/07/2018  22:20        15 375 923 getPosesRpt2.pkl
12/07/2018  20:01            27 586 Perf_RT.ipynb
10/07/2018  17:39             6 567 Tutorial_Portfolio_Vizualisation.ipynb
16/07/2018  15:05             7 121 Tuto_Attrib-Copy1.ipynb
17/07/2018  13:27            17 270 Tuto_Attrib.ipynb
10/07/2018  17:41            15 366 vizualise.py
               8 File(s)     28 854 378 bytes
               3 Dir(s)  29 843 013 632 bytes free


In [53]:
!jupyter nbconvert Tuto_Attrib.ipynb --to html

[NbConvertApp] Converting notebook Tuto_Attrib.ipynb to html
[NbConvertApp] Writing 280972 bytes to Tuto_Attrib.html
